# 线性回归的简洁实现
利用pytorch开源框架，我们能够用深度学习库中的诸多组件代替我们自定义的函数。
## 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
import MyTool

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = MyTool.synthetic_data(true_w, true_b, 1000)

load_data_fashion_mnist is defined


## 读取数据集
我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True): 
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用data_iter的方式与我们在上一节中使用data_iter函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。与上一节不同，这里我们使用iter构造Python迭代器，并使用next从迭代器中获取第一项。

In [3]:
next(iter(data_iter))

[tensor([[ 1.6307, -1.2781],
         [-1.2649, -0.8718],
         [-0.7719,  0.6059],
         [ 0.3302, -0.5316],
         [-1.8238, -1.3007],
         [ 0.1917, -0.0892],
         [-0.7161, -0.0784],
         [ 0.5811,  1.6245],
         [ 0.9908,  1.1675],
         [-1.2787,  1.5004]]),
 tensor([[11.8210],
         [ 4.6360],
         [ 0.5905],
         [ 6.6757],
         [ 4.9834],
         [ 4.9087],
         [ 3.0141],
         [-0.1747],
         [ 2.2044],
         [-3.4604]])]

## 定义模型
对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的pipeline”。
在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [4]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

## 初始化模型参数
正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数
计算均方误差使用的是MSELoss类，也称为平方L_2范数。 默认情况下，它返回所有样本损失的平均值。

In [6]:
loss = nn.MSELoss()

## 定义优化算法
小批量随机梯度下降算法是一种优化神经网络的标准工具， PyTorch在optim模块中实现了该算法的许多变种。 当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 小批量随机梯度下降只需要设置lr值，这里设置为0.03。

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练
每个迭代周期里，我们将完整遍历一次数据集（train_data）， 不停地从中获取一个小批量的输入和相应的标签。 对于每一个小批量，我们会进行以下步骤:
- 通过调用net(X)生成预测并计算损失l（前向传播）。
- 通过进行反向传播来计算梯度。
- 通过调用优化器来更新模型参数。

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # 前向传播
        trainer.zero_grad()  # 梯度清零
        l.backward()  # 反向传播
        trainer.step()  # 更新参数
    l = loss(net(features), labels)  # 完整数据集的损失
    print(f'epoch {epoch + 1}, loss {l:f}')  # 打印损失值

epoch 1, loss 0.000168
epoch 2, loss 0.000099
epoch 3, loss 0.000097


我们同样可以通过比较训练得到的w, b与true_w, true_b来评估训练结果。

In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-3.7527e-04,  3.6478e-05])
b的估计误差： tensor([0.0003])
